## Imports

In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
import math
import copy

import torch
import torchvision 
import torch.optim as optim
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from skimage.restoration import denoise_nl_means, estimate_sigma

### import utilities  from cyphercat repo
sys.path.insert(0, 'Utils')
import models
from train import *
from metrics import * 



## Hyperparameters

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="cuda:0"
import pycuda.driver as cuda
cuda.init()

In [3]:
n_epochs = 100
batch_size = 32
lr = 0.01

## Load AT&t faces dataset

In [4]:
data_dir = "../DATA/att_faces/orl_faces"

img_paths = []
for p in os.listdir(data_dir):
    for i in os.listdir(os.path.join(data_dir, p)):
        img_paths.append(os.path.join(data_dir, p, i))

people = []
people_to_idx = {}

k=0
for i in img_paths:
    name = i.split('/')[4]
    if name not in people_to_idx:
        people.append(name)
        people_to_idx[name] = k
        k += 1


In [5]:
img_paths = np.random.permutation(img_paths)
att_size = len(img_paths)
att_train_size = int(0.7 * att_size)

att_train_list = img_paths[:att_train_size]
att_test_list = img_paths[att_train_size:]

In [6]:
class ATTdataset(Dataset): 
    def __init__(self, file_list, class_to_label, transform=None): 
        self.file_list = file_list
        self.transform = transform
        self.people_to_idx = class_to_label

    def __len__(self): 
        return len(self.file_list)

    def __getitem__(self, idx): 
        image = np.empty((112, 92, 1), dtype=np.uint8)
        img_path = self.file_list[idx]
        im = io.imread(img_path)
        im = im.reshape(112,92)
        image[:,:,0]= im
#         image[:,:,1]= im
#         image[:,:,2]= im

        label = self.people_to_idx[img_path.split('/')[4]]
        
        if self.transform is not None: 
            image = self.transform(image)
        return image, label

## Data Augmentation

In [7]:
train_transform = torchvision.transforms.Compose([
    # TODO try different data tranforms
    
    # Convert numpy.ndarray to tensor.
    torchvision.transforms.ToTensor(),
    
    # Normalize a tensor image with mean and standard deviation. Given mean: (M1,...,Mn) and std: (S1,..,Sn)
    torchvision.transforms.Normalize((0.5,0.5,0.5), (0.5, 0.5, 0.5))
])

test_transform = torchvision.transforms.Compose([
    # TODO try different data tranforms
    
    # Convert numpy.ndarray to tensor.
    torchvision.transforms.ToTensor(),
    
    # Normalize a tensor image with mean and standard deviation. Given mean: (M1,...,Mn) and std: (S1,..,Sn)
    torchvision.transforms.Normalize((0.5,0.5,0.5), (0.5, 0.5, 0.5))
])


In [8]:
# train/test split
trainset = ATTdataset(att_train_list, people_to_idx, transform=train_transform)
testset = ATTdataset(att_test_list, people_to_idx, transform=test_transform)
sampleset = ATTdataset(att_test_list, people_to_idx, transform=torchvision.transforms.ToTensor())

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)
sampleloader = torch.utils.data.DataLoader(sampleset, batch_size=1, shuffle=False, num_workers=2)
                                           

In [9]:
# helper function to un-normalize and plot image
def imshow(img):
    img = np.array(img)
    img = img / 2 + 0.5
    img = np.moveaxis(img, 0, -1)
    plt.imshow(img)

# display sample from dataset 
imgs, labels = iter(trainloader).next()
imshow(torchvision.utils.make_grid(imgs)) 
print(labels)
print(imgs.min())
print(imgs.max())
print(imgs.shape)

## Model Architecture

In [10]:
# 1 hidden layer only
class mlp(nn.Module): 
    def __init__(self, n_in=112*92, n_out=40, n_hidden=3000): 
        super(mlp, self).__init__()
        # Applies a linear transformation to the incoming data: y = xA^T + b
        self.hidden1 = nn.Linear(n_in, n_out)        

    def forward(self, x):

        x = x.view(-1,112*92)
        out = self.hidden1(x)
        return out



In [11]:
# determine device to run network on (runs on gpu if available)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = mlp().to(device)
net.apply(models.weights_init)

net2 = mlp().to(device)
net2.apply(models.weights_init)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=lr)
optimizer2 = optim.SGD(net2.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)


ww0 = copy.deepcopy(net2.hidden1.weight.data)
bb = copy.deepcopy(net2.hidden1.bias.data[:])
im0 = ww0[30].reshape(112,92)


## training

In [12]:
train(net, trainloader, testloader, optimizer, criterion, n_epochs, verbose=False)


In [13]:
train(net2, trainloader, testloader, optimizer2, criterion, n_epochs, verbose=False)

In [17]:
ww = net2.hidden1.weight.data[:]
bb = net2.hidden1.bias.data[:]
im0CPU = im0.cpu().detach().numpy()

im_train = ww[20,:].squeeze().cpu().detach().numpy().reshape(112,92)
plt.imshow(im0CPU,cmap='gray')
plt.show()

plt.imshow(im_train,cmap='gray')
plt.show()

plt.imshow(im_train-im0CPU,cmap='gray')
plt.show()

randimg = torch.randn(im_train.shape).cpu().detach().numpy()/100
plt.imshow(im_train-randimg,cmap='gray')
plt.show()


## Gradient Ascent



In [ ]:
from torch.autograd import Variable

def ga(image, net, iterations, lr,category):        
    category = Variable(torch.cuda.LongTensor([category]))
    input = Variable(image.unsqueeze(0).cuda(), requires_grad=True)
    losses = []
    grad0 = np.zeros((112,92))
    for j in range(iterations):
        for i in range(1):

            net.zero_grad()
            
#             out = nn.functional.softmax(net(input), dim=1)
            out = net(input)
            loss= -out.take(category)#+reg_alpha
            
            loss.backward()
            input.data = input.data - lr * input.grad.data
            input.grad.zero_()
        losses.append(loss.data)
#         print(j)
        grad0=input.grad.data.squeeze().cpu().detach().numpy()
        input.grad.data = input.grad.data - input.grad.data
#         plt.imsave(f"Results/2Fred {j}.png",input.squeeze().cpu().detach().numpy(),cmap = 'gray',vmin =-1 , vmax = 1,format='png')
    plt.plot(losses)
    plt.show()

    
    return input
